# Clean Text 

In [1]:
import pandas as pd
import os
import re

In [2]:
dt = pd.read_csv(os.path.join("/Users/Project/RawData", 'HotelsReviews.csv'))

In [3]:
dt.head()

,reviewdate,star,user_info,Hotel Name,Street,City,State,Zip,reviews
0,9/4/05,5,Sharon-JoeJr,1 Church Street The White House Inn,1 Church Street,Belfast,ME,4915,"Comfortable elegance, Wonderful Hosts Terry a..."
1,8/6/07,5,UKMaggiemay,1 Church Street The White House Inn,1 Church Street,Belfast,ME,4915,"Superb, we stayed at this wonderful BB 3 years..."
2,8/24/07,5,MattMcLean,1 Church Street The White House Inn,1 Church Street,Belfast,ME,4915,"Fantastic Experience, We just returned from a ..."
3,6/3/09,5,PWalk,1 Church Street The White House Inn,1 Church Street,Belfast,ME,4915,"A Lovely Time, What a lovely time The hosts ar..."
4,6/11/09,5,swimmerbee,1 Church Street The White House Inn,1 Church Street,Belfast,ME,4915,"Perfect, We were originally going to stay in P..."


In [ ]:
import nltk.data
nltk.download()

In [ ]:
from nltk.corpus import stopwords

def text2wordlist( text):
    # remove non-letters
    txt=re.sub("[^a-zA-Z]"," ",text)
    # convert to lower case
    words=txt.lower().split()
    # remove stop words
    stop_wds=set(stopwords.words("english"))
    stop_wds.add('nan')
    words = [w for w in words if not w in stop_wds]
    return words

In [ ]:
import nltk.data
tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')

def text2sentences( text, tokenizer):
    # the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(text.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append( text2wordlist( raw_sentence))
    return sentences

In [ ]:
t1='...Beautiful hotel. Wonderful for any occasion.'
tokenizer.tokenize(t1.strip())

In [ ]:
hotel=dt['Hotel Name'][1]
texts=dt['reviews']
HotelID=[hotel]
words_i=[]

for i in range(len(texts)):
    if dt['Hotel Name'][i]==hotel:
        words_i.append(text2sentences(texts[i],tokenizer))
    else:
        print('now for hotel: '+str(hotel))
        HotelID.append(dt['Hotel Name'][i])
        hotel=dt['Hotel Name'][i]

In [ ]:
# write id in order 
with open('hotels_ids.txt', 'w') as file:
     file.write(str(HotelID))
        
# write clean text in order       
with open('hotels_review.txt', 'w') as file:
    file.write(str(words_i[1]))        
for i in range(1,len(words_i)):
    with open('hotels_review.txt', 'd') as file:
         file.write(str(words_i[i]))

In [ ]:
import re
with open('hotels_review.txt', 'r') as file:
    for i in file:
        txt=re.sub("\[","{",i) 
txt=re.sub("]","}",txt) 

# Train Model

In [ ]:
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [ ]:
# set values for various parameters
num_features = 50    # Word vector dimensionality                      
min_word_count = 100  # Minimum word count                        
num_workers = 5       # Number of threads to run in parallel
context = 5          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [ ]:
# input
sentences=[]
for i in range(len(words_i)):
    sentences += words_i[1]

In [ ]:
len(sentences)

In [ ]:
# train the model (this will take some time)
import gensim
from gensim.models import word2vec
print("   Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

In [ ]:
model.most_similar("service")

In [ ]:
model.most_similar("beds")

In [ ]:
model.doesnt_match("man night love kitchen beds lots".split())

In [ ]:
model.wv.syn0.shape

In [ ]:
model.init_sims(replace=True)
model_name = "textmining_200d100"
model.save(model_name)

In [ ]:
# model = Word2Vec.load("textmining_200d100")

# K mean

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import time

In [ ]:
model.wv.syn0.shape[0]

In [ ]:
num_clusters

In [ ]:
start = time.time()

num_clusters = int(model.wv.syn0.shape[0]/10)
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

end = time.time()
elapsed = end - start

print("number of clusters: "+str(num_clusters))
print("Time taken for K Means clustering: ", elapsed, "seconds.")

In [ ]:
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [ ]:
word_centroid_map